In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import astropy.stats as st

from calibTools import *

In [2]:
table   = np.genfromtxt('corrected_mags_linewidth_all.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)


Au_e  = getReddening_err(table, band1='u', band2='w2')
Ag_e  = getReddening_err(table, band1='g', band2='w2')
Ar_e  = getReddening_err(table, band1='r', band2='w2')
Ai_e  = getReddening_err(table, band1='i', band2='w2')
Az_e  = getReddening_err(table, band1='z', band2='w2')
Aw1_e  = getReddening_err(table, band1='w1', band2='w2')
Aw2_e  = getReddening_err(table, band1='w2', band2='w2')


catal   = np.genfromtxt('ESN_HI_catal_all.csv' , delimiter=',', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_catal   = catal['pgc']
R50u_catal  = catal['R50_u']
R50g_catal  = catal['R50_g']
R50r_catal  = catal['R50_r']
R50i_catal  = catal['R50_i']
R50z_catal  = catal['R50_z']
R50w1_catal = catal['R50_w1']
R50w2_catal = catal['R50_w2']
Sba_catal   = catal['Sba']
Wba_catal   = catal['Wba']


pgc    = table['pgc']
Name   = table['Name']
inc    = table['inc']
inc_e  = table['inc_e']
Wmx    = table['Wmx']
eWmx   = table['eWmx']
logWimx = table['logWimx']
logWimx_e  = table['logWimx_e']
m21    = table['m21']
m21_e  = table['m21_e']
uu     = table['u']
gg     = table['g']
rr     = table['r']
ii     = table['i']
zz     = table['z']
w1     = table['w1']
w2     = table['w2']


u_     = table['u_']
g_     = table['g_']
r_     = table['r_']
i_     = table['i_']
z_     = table['z_']
w1_    = table['w1_']
w2_    = table['w2_']

N = len(pgc)
R50u  = np.zeros(N)
R50g  = np.zeros(N)
R50r  = np.zeros(N)
R50i  = np.zeros(N)
R50z  = np.zeros(N)
R50w1 = np.zeros(N)
R50w2 = np.zeros(N)
Sba   = np.zeros(N)
Wba   = np.zeros(N)

u_s   = table['u_sss']
g_s   = table['g_sss']
r_s   = table['r_sss']
i_s   = table['i_sss']
z_s   = table['z_sss']
w1_s  = table['w1_sss']
w2_s  = table['w2_sss']


for i, idd in enumerate(pgc):
    if idd in pgc_catal:
        indx,   = np.where(pgc_catal==idd)
        R50u[i]  = R50u_catal[indx][0]
        R50g[i]  = R50g_catal[indx][0]
        R50r[i]  = R50r_catal[indx][0]
        R50i[i]  = R50i_catal[indx][0]
        R50z[i]  = R50z_catal[indx][0]
        R50w1[i] = R50w1_catal[indx][0]
        R50w2[i] = R50w2_catal[indx][0]
        Sba[i]   = Sba_catal[indx][0]
        Wba[i]   = Wba_catal[indx][0]

        
gamma_u     = table['gamma_u']
gamma_g     = table['gamma_g']
gamma_r     = table['gamma_r']
gamma_i     = table['gamma_i']
gamma_z     = table['gamma_z']
gamma_w1    = table['gamma_w1']
gamma_w2    = table['gamma_w2']
w2p_     = table['W2p_']
gamma_u_p     = table['gamma_p_u']
gamma_g_p     = table['gamma_p_g']
gamma_r_p     = table['gamma_p_r']
gamma_i_p     = table['gamma_p_i']
gamma_z_p     = table['gamma_p_z']
gamma_w1_p     = table['gamma_p_w1']
gamma_w2_p     = table['gamma_p_w2']
Au_p     = table['A_p_u']
Ag_p     = table['A_p_g']
Ar_p     = table['A_p_r']
Ai_p     = table['A_p_i']
Az_p     = table['A_p_z']
Aw1_p     = table['A_p_w1']
Aw2_p     = table['A_p_w2']
P1_W2     = table['P0_w2']
eP1_W2     = table['P0_w2_e']
P1_W2p     = table['P0_w2p']
P1_W1     = table['P0_w1']
Qs     = table['Sqlt']
Qw     = table['Wqlt']    
R_source     = table['R_source']
     
    

In [10]:

indx = np.argsort(pgc)

Nrow = 0
for i in indx:
    
    row = ''
    idd = pgc[i]
    
    j, = np.where(pgc == idd)
    j = j[0]
    
    if True:
        Nrow+=1
        
        row += str(idd) + ' & '
        row += Name[j].strip()  + ' & '
        
        row += '%.0f'%inc[j] + '$\pm$' + '%.0f'%inc_e[j] + ' & '
        row += '%.0f'%Wmx[j] + '$\pm$' + '%.0f'%eWmx[j] + ' & '
        row += '%.3f'%logWimx[j] + '$\pm$' + '%.3f'%logWimx_e[j] + ' & '
        row += '%.3f'%m21[j] + '$\pm$' + '%.3f'%m21_e[j] + ' & '
        
        row += '%.2f'%uu[j] + ' & '
        row += '%.2f'%gg[j] + ' & '
        row += '%.2f'%rr[j] + ' & '
        row += '%.2f'%ii[j] + ' & '
        row += '%.2f'%zz[j] + ' & '
        row += '%.2f'%w1[j] + ' & '
        row += '%.2f'%w2[j] + ' & '
        
        row += '%.2f'%Sba[j] + ' & '
        row += '%.2f'%Wba[j] + ' & '

        row += '%.2f'%R50u[j] + ' & '
        row += '%.2f'%R50g[j] + ' & '
        row += '%.2f'%R50r[j] + ' & '
        row += '%.2f'%R50i[j] + ' & '
        row += '%.2f'%R50z[j] + ' & '
        row += '%.2f'%R50w1[j] + ' & '
        row += '%.2f'%R50w2[j] + ' \\\\ '        
        
        
        print row
        
    
    if Nrow==15:
        break
    

2 & UGC12889 & 52$\pm$4 & 437$\pm$16 & 2.744$\pm$0.029 & 15.713$\pm$0.185 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 11.86 & 12.49 & 0.00 & 0.66 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.51 & 0.51 \\ 
4 & PGC000004 & 85$\pm$2 & 154$\pm$4 & 2.189$\pm$0.011 & 16.688$\pm$0.076 & 17.57 & 16.43 & 15.91 & 15.57 & 15.42 & 16.06 & 16.42 & 0.24 & 0.33 & 0.25 & 0.23 & 0.21 & 0.23 & 0.27 & 0.27 & 0.33 \\ 
12 & PGC000012 & 82$\pm$3 & 400$\pm$19 & 2.606$\pm$0.021 & 16.071$\pm$0.185 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 13.60 & 14.20 & 0.22 & 0.36 & 0.43 & 1.24 & 1.38 & -0.02 & 1.40 & 0.25 & 0.25 \\ 
16 & PGC000016 & 65$\pm$4 & 297$\pm$14 & 2.515$\pm$0.025 & 17.207$\pm$0.185 & 16.08 & 14.83 & 14.21 & 13.86 & 13.61 & 13.98 & 14.67 & 0.52 & 0.64 & 0.22 & 0.21 & 0.19 & 0.19 & 0.19 & 0.21 & 0.20 \\ 
55 & UGC12898 & 80$\pm$3 & 179$\pm$10 & 2.260$\pm$0.025 & 15.757$\pm$0.076 & 16.82 & 16.08 & 15.71 & 15.50 & 15.40 & 16.18 & 16.60 & 0.32 & 0.46 & 0.28 & 0.23 & 0.23 & 0.22 & 0.22 & 0.22 & 0.26 \\ 
68 & ESO538-017 & 5

In [11]:
indx = np.argsort(pgc)

Nrow = 0
for i in indx:
    
    row = ''
    idd = pgc[i]
    
    j, = np.where(pgc == idd)
    j = j[0]
    
    if True:
        Nrow+=1

        row += str(idd) + ' & '

        row += '%.2f'%u_[j]  + ' & '
        row += '%.2f'%g_[j]  + ' & '
        row += '%.2f'%r_[j]  + ' & '
        row += '%.2f'%i_[j]  + ' & '
        row += '%.2f'%z_[j]  + ' & '
        row += '%.2f'%w1_[j] + ' & '
        row += '%.2f'%w2_[j] + ' & '


        row += '%.2f'%(u_[j]-u_s[j]) + '$\pm$' + '%.2f'%Au_e[j]  + ' & '
        row += '%.2f'%(g_[j]-g_s[j]) + '$\pm$' + '%.2f'%Ag_e[j]  + ' & '
        row += '%.2f'%(r_[j]-r_s[j]) + '$\pm$' + '%.2f'%Ar_e[j]  + ' & '
        row += '%.2f'%(i_[j]-i_s[j]) + '$\pm$' + '%.2f'%Ai_e[j]  + ' & '
        row += '%.2f'%(z_[j]-z_s[j]) + '$\pm$' + '%.2f'%Az_e[j]  + ' & '
        row += '%.2f'%(w1_[j]-w1_s[j]) + '$\pm$' + '%.2f'%Aw1_e[j] + ' & '
        row += '%.2f'%(w2_[j]-w2_s[j]) + '$\pm$' + '%.2f'%Aw2_e[j] + ' & '        


        row += '%.2f'%u_s[j]  + ' & '
        row += '%.2f'%g_s[j]  + ' & '
        row += '%.2f'%r_s[j]  + ' & '
        row += '%.2f'%i_s[j]  + ' & '
        row += '%.2f'%z_s[j]  + ' \\\\ '
               

        print row
        
    
    if Nrow==15:
        break
    

2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 11.90 & 12.53 & 0.00$\pm$0.04 & 0.00$\pm$0.03 & 0.00$\pm$0.02 & 0.00$\pm$0.02 & 0.00$\pm$0.01 & 0.01$\pm$0.00 & 0.01$\pm$0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\ 
4 & 17.11 & 16.11 & 15.69 & 15.41 & 15.30 & 16.08 & 16.44 & 0.81$\pm$0.12 & 0.52$\pm$0.08 & 0.36$\pm$0.06 & 0.29$\pm$0.05 & 0.23$\pm$0.04 & 0.01$\pm$0.00 & 0.00$\pm$0.00 & 16.30 & 15.59 & 15.33 & 15.12 & 15.07 \\ 
12 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 13.64 & 14.24 & 0.00$\pm$0.19 & 0.00$\pm$0.14 & 0.00$\pm$0.11 & 0.00$\pm$0.09 & 0.00$\pm$0.07 & 0.05$\pm$0.01 & 0.01$\pm$0.01 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\ 
16 & 15.86 & 14.69 & 14.12 & 13.79 & 13.56 & 14.02 & 14.71 & 0.46$\pm$0.08 & 0.33$\pm$0.06 & 0.26$\pm$0.05 & 0.21$\pm$0.04 & 0.16$\pm$0.03 & 0.03$\pm$0.00 & 0.01$\pm$0.00 & 15.40 & 14.36 & 13.86 & 13.58 & 13.40 \\ 
55 & 16.53 & 15.90 & 15.58 & 15.42 & 15.33 & 16.21 & 16.63 & 0.59$\pm$0.10 & 0.37$\pm$0.06 & 0.26$\pm$0.04 & 0.20$\pm$0.04 & 0.16$\pm$0.03 & 0.01$\pm$0.00 & 0.00$

In [13]:
indx = np.argsort(pgc)

Nrow = 0
for i in indx:
    
    row = ''
    idd = pgc[i]
    
    j, = np.where(pgc == idd)
    j = j[0]
    
    if True:
        Nrow+=1

        row += str(idd) + ' & '
        
        row += '%.2f'%w1_s[j] + ' & '
        row += '%.2f'%w2_s[j] + ' & ' 
        
        
        row += str(w2p_[j]) + ' & '

        row += '%.2f'%gamma_u[j]  + ' & '
        row += '%.2f'%gamma_g[j]  + ' & '
        row += '%.2f'%gamma_r[j]  + ' & '
        row += '%.2f'%gamma_i[j]  + ' & '
        row += '%.2f'%gamma_z[j]  + ' & '
        row += '%.2f'%gamma_w1[j]  + ' & '
        row += '%.2f'%gamma_w2[j]  + ' & '
        
        
        row += '%.2f'%gamma_u_p[j]  + ' & '
        row += '%.2f'%gamma_g_p[j]  + ' & '
        row += '%.2f'%gamma_r_p[j]  + ' & '
        row += '%.2f'%gamma_i_p[j]  + ' & '
        row += '%.2f'%gamma_z_p[j]  + ' & '
        row += '%.2f'%gamma_w1_p[j]  + ' & '
        row += '%.2f'%gamma_w2_p[j]  + ' \\\\ '
        


        print row
        
    
    if Nrow==15:
        break
    

2 & 11.89 & 12.52 & 12.53 & 1.06 & 0.70 & 0.51 & 0.37 & 0.20 & 0.05 & 0.02 & 1.06 & 0.70 & 0.51 & 0.37 & 0.20 & 0.05 & 0.02 \\ 
4 & 16.07 & 16.44 & 16.55 & 0.79 & 0.50 & 0.34 & 0.28 & 0.22 & 0.01 & 0.00 & 0.79 & 0.50 & 0.34 & 0.28 & 0.22 & 0.01 & 0.00 \\ 
12 & 13.59 & 14.23 & 14.24 & 1.24 & 0.90 & 0.71 & 0.60 & 0.47 & 0.07 & 0.02 & 1.24 & 0.90 & 0.71 & 0.60 & 0.47 & 0.07 & 0.02 \\ 
16 & 13.99 & 14.70 & 14.43 & 1.23 & 0.89 & 0.70 & 0.58 & 0.44 & 0.07 & 0.02 & 1.23 & 0.88 & 0.69 & 0.57 & 0.43 & 0.07 & 0.02 \\ 
55 & 16.20 & 16.63 & 16.81 & 0.78 & 0.49 & 0.34 & 0.27 & 0.22 & 0.01 & 0.00 & 0.77 & 0.47 & 0.32 & 0.26 & 0.21 & 0.01 & 0.00 \\ 
68 & 15.03 & 15.54 & 15.5 & 1.16 & 0.84 & 0.66 & 0.56 & 0.46 & 0.06 & 0.02 & 1.17 & 0.84 & 0.67 & 0.57 & 0.46 & 0.06 & 0.02 \\ 
70 & 13.54 & 14.11 & 14.42 & 1.11 & 0.79 & 0.62 & 0.52 & 0.43 & 0.05 & 0.02 & 1.08 & 0.76 & 0.59 & 0.50 & 0.42 & 0.04 & 0.01 \\ 
76 & 13.23 & 13.81 & 13.86 & 1.24 & 0.90 & 0.71 & 0.59 & 0.46 & 0.07 & 0.02 & 1.23 & 0.89 & 0.70 & 0

In [14]:
indx = np.argsort(pgc)

Nrow = 0
for i in indx:
    
    row = ''
    idd = pgc[i]
    
    j, = np.where(pgc == idd)
    j = j[0]
    
    if True:
        Nrow+=1

        row += str(idd) + ' & '
        

        
        row += '%.2f'%Au_p[j]  + ' & '
        row += '%.2f'%Ag_p[j]  + ' & '
        row += '%.2f'%Ar_p[j]  + ' & '
        row += '%.2f'%Ai_p[j]  + ' & '
        row += '%.2f'%Az_p[j]  + ' & '
        row += '%.2f'%Aw1_p[j]  + ' & '
        row += '%.2f'%Aw2_p[j]  + ' & '


        row += '%.2f'%P1_W2[j] + '$\pm$' + '%.2f'%eP1_W2[j]  + ' & '
        row += '%.2f'%P1_W2p[j]  + ' & '
        row += '%.2f'%P1_W1[j]  + ' & '
        row += '%d'%Qs[j]  + ' & '
        row += '%d'%Qw[j]  + ' & '
        row += '%d'%R_source[j]  + ' \\\\ '

        
        print row
        
    
    if Nrow==15:
        break
    

2 & 0.22 & 0.15 & 0.11 & 0.08 & 0.04 & 0.01 & 0.00 & 2.02$\pm$0.14 & 2.02 & 2.14 & 0 & 5 & 2 \\ 
4 & 0.82 & 0.52 & 0.36 & 0.29 & 0.24 & 0.01 & 0.00 & -2.75$\pm$0.10 & -2.73 & -2.69 & 5 & 4 & 2 \\ 
12 & 1.04 & 0.76 & 0.60 & 0.51 & 0.40 & 0.05 & 0.02 & 0.56$\pm$0.15 & 0.56 & 0.66 & 0 & 5 & 2 \\ 
16 & 0.46 & 0.33 & 0.26 & 0.21 & 0.16 & 0.02 & 0.01 & 0.92$\pm$0.15 & 1.02 & 1.04 & 5 & 5 & 2 \\ 
55 & 0.58 & 0.36 & 0.24 & 0.20 & 0.16 & 0.01 & 0.00 & -2.83$\pm$0.12 & -3.03 & -2.73 & 5 & 4 & 2 \\ 
68 & 0.31 & 0.22 & 0.18 & 0.15 & 0.12 & 0.01 & 0.00 & -0.35$\pm$0.18 & -0.30 & -0.35 & 5 & 5 & 2 \\ 
70 & 1.58 & 1.19 & 0.96 & 0.83 & 0.71 & 0.04 & 0.01 & -0.74$\pm$0.14 & -0.91 & -0.69 & 5 & 5 & 2 \\ 
76 & 0.52 & 0.38 & 0.30 & 0.25 & 0.19 & 0.03 & 0.01 & 0.75$\pm$0.09 & 0.89 & 0.88 & 5 & 5 & 2 \\ 
92 & 0.59 & 0.37 & 0.26 & 0.21 & 0.17 & 0.01 & 0.00 & -2.78$\pm$0.11 & -2.78 & 0.00 & 5 & 0 & 0 \\ 
94 & 1.14 & 0.76 & 0.53 & 0.44 & 0.36 & 0.01 & 0.00 & -2.91$\pm$0.17 & -2.91 & -2.79 & 0 & 4 & 2 \\ 
96 & 